## Port.hu ULMFiT classifier 

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.text import *

In [3]:
path = Path("/home/jupyter/learn-ai/data")
filename = 'port_hu_mozes.csv'

In [4]:
df = pd.read_csv(path/filename)
df.head()

,értékelés,szöveg
0,pozitív,Teljesen elment a kedvem a lélekvándorlástól. ...
1,negatív,"Kivancsi vagyok, sikerult-e mar a TV2 csoportn..."
2,negatív,"Kíváncsi vagyok, ezúttal vajon lesz-e felirat ..."
3,negatív,"Ez így igaz, másrészt a csupa egyéni véleménye..."
4,pozitív,Ez egy 5 pontos film maximum. Sok szart felpon...


In [5]:
bs = 50

In [6]:
data_bunch = (TextList
           .from_csv(path, filename, cols='szöveg')
           .split_by_rand_pct(0.15)
           .label_for_lm()
           .databunch(bs=bs))

In [7]:
data_bunch.save('data_bunch.pkl')

In [8]:
data_bunch.show_batch()

idx,text
0,"e mar a xxup tv2 xxunk xxunk a hindi xxunk xxunk , eddig ugyanis xxunk lattam valamelyik xxunk , erre xxunk sem vetettek . xxmaj vagy esetleg azt gondoljak , mindenki perfekt xxunk xxmaj xxunk ? ? ! xxbos xxmaj kíváncsi vagyok , ezúttal vajon lesz - e felirat a hindi nyelvû jeleneteknél , vagy a xxunk még mindig azt gondolja , hogy xxmaj magyarországon mindenki perfekt e xxunk xxrep"
1,xxmaj ez a foto xxbos xxmaj cs xxrep 5 õ xxmaj mindenki xxrep 7 ! xxmaj én ugy tudom h tom hardy fogja játszani az egyik fõgonoszt mégpedig vmi xxunk talán xxunk . xxmaj meg még benne lesz xxunk lánya meg a xxunk kb ennyit tudok xxrep 5 ! xxmaj de itt egy kép amit mindenkinek látnia kell fél hivatalos foto : xxbos xxmaj azért én kicsit izgulok . xxmaj
2,xxup d. . xxmaj szóval nekem bejött 8 / 10 \n xxunk akartam rá adni de véletlenül a 7-esre kattintottam :) \n \n xxmaj és eddig a sötét lovag xxunk csak amerikába ) a 29 . helyen áll . xxmaj eddig $ xxunk folyt be a xxunk : ) maga mögé xxunk a transformerst ! xxup usa - ba pedig már a 3 . : ) ( remélem
3,"ismert térben / idõben zajlanak az események , így aztán simán xxunk furcsaságok . xxmaj én simán el tudom fogadni azt , vagy úgy mondom , legalább is én így gondolom , hogy a xxmaj gargantua nem csupán egy sima fekete lyuk a filmben , hanem egyfajta jövõbeliek által „ xxunk ” fekete lyuk . xxmaj ezért xxunk benne a hiperkocka , ezért nem xxunk fel xxmaj coopert és xxup"
4,"olyan , mintha xxmaj neil xxmaj xxunk xxunk volna a xxunk , egy hirtelen xxunk után . \n \n \n \n ' xxmaj továbbá a film tudományosan egy jövõt feltételez még akkor is ha a világ épp xxunk szorul a xxunk . xxmaj azért mert a xxunk xxunk erõltetik , az nem jelent technológiai xxunk . ' \n \n xxmaj majd a xxunk ( ha létezik ) xxunk"


In [9]:
data_path = Config.data_path();data_path

PosixPath('/home/jupyter/.fastai/data')

In [10]:
pretrained_model_fnames = [data_path/"ulmfit/hulm", data_path/"ulmfit/hulm_vocab"]

In [11]:
learn_model = language_model_learner(data_bunch, AWD_LSTM, drop_mult=1.0, pretrained_fnames=pretrained_model_fnames)

In [12]:
learning_rate = 1e-3

In [13]:
learn_model.unfreeze()
learn_model.fit_one_cycle(6, learning_rate, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,5.005511,4.667676,0.249451,01:34
1,4.642906,4.379677,0.265791,01:33
2,4.452706,4.256414,0.273016,01:33
3,4.328825,4.196009,0.277500,01:33
4,4.255821,4.171099,0.279978,01:33
5,4.210202,4.166823,0.280143,01:33


In [14]:
learn_model.save("hu_port")
learn_model.save_encoder("hu_port_enc")

## classifier

In [15]:
data_clas = (TextList.from_df(df, path, vocab=data_bunch.vocab, cols='szöveg')
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='értékelés')
    .databunch(bs=bs, num_workers=1, backwards=True))

data_clas.save("hu_text_clas_bwd")


In [16]:
data_clas = load_data(path, "hu_text_clas_bwd", bs=bs, num_workers=1, backwards=True)

In [17]:
from sklearn.metrics import f1_score

@np_func
def f1(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1))

In [27]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy])
learn_c.load_encoder("hu_port_enc")
learn_c.freeze()

In [28]:
learn_c.unfreeze()

In [29]:
lr=2e-2

In [30]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.331013,0.374651,0.872822,02:05
1,0.243282,0.812667,0.878484,02:09


In [31]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.221645,0.348334,0.885017,01:56
1,0.205364,0.376566,0.884582,01:40


In [32]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.212338,0.293746,0.891768,01:58
1,0.183678,0.309016,0.887413,02:08


In [ ]:
learn_c.unfreeze()
learn_c.fit_one_cycle(1, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time


In [25]:
learn_c.save("hu_port_classifier")